## MODOAP - Detection and extraction of illustrations inside document pages 

This script detects and extracts illustrations inside newspapers, magazines, or school books pages.
It requires :

  - a model file from a previous training
  - a dataset which consists of a folder containing image files representing document pages

This script implements the configuration which can be found on https://github.com/matterport/Mask_RCNN

**The runtime environment must be set to GPU : Execution -> Change runtime environment -> GPU**

In [ ]:
#@title ##  Preparation

#@markdown ### Connecting to Google Drive and setting up the algotrithm
#@markdown ##### Run this cell then enter a Google Account


import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")


%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install numpy
  !pip install scipy
  !pip install Pillow
  !pip install cython
  !pip install  matplotlib
  !pip install scikit-image
  !pip install tensorflow>=1.3.0

  !pip install keras>=2.0.8

  !pip install opencv-python

  !pip install h5py==2.10.0
  !pip install imgaug
  !pip install IPython[all]
  !python setup.py install
  !cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py
  !sed -i -- 's/balloon/illustration/g' illustration.py
  !sed -i -- 's/Balloon/Illustration/g' illustration.py
 
else : print("L'algorithme est déjà téléchargé")

%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5

%cd /root/Mask_RCNN/

import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob

ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

sys.path.append(ROOT_DIR) 
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration


In [ ]:
#@markdown ## Detect illustrations in dataset
#@markdown #### This cell generates a file called annotations_illustration.json in the dataset folder

#@markdown ### Enter a path to a folder containing the pages of the documents :
dataset_dir = "" #@param {type:"string"}
#@markdown Path example:
#@markdown /content/drive/My Drive/Corpus/

#@markdown ---

#@markdown ### Enter a path to a model file (*.h5) from a previous training :
model_path = "" #@param {type:"string"}
#@markdown Path example:
#@markdown /content/drive/MyDrive/Outils_Modoap/Detection_Illustrations/Poids/illustration20210303T1457/mask_rcnn_illustration_0200.h5

#@markdown ---
#@markdown ### Activate results vizualisation (longer calculation to check the results) :
visualisation = "oui" #@param ["oui", "non"]

dossier_corpus = dataset_dir
fichierpoids = model_path

IMAGE_DIR = dossier_corpus
config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

warnings.filterwarnings("ignore")
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)
model.load_weights(fichierpoids, by_name=True)

corpus = [image for image in glob.glob(dossier_corpus+"/*.*") if not image.split(".")[-1].endswith("json")]
class_names = ['BG', 'illustration']
import cv2
import json
dicocorpus = {}
print("Launching detection on {} pages".format(len(corpus)))

compteur = 1
for cheminfichier in corpus :
  print("Page {} on {}".format(compteur, len(corpus)))
  nomfichier = str(cheminfichier.split("/")[-1])
  image = cv2.imread(cheminfichier)
  results = model.detect([image], verbose=0)
  r = results[0]
  listeobjets = [vecteur for vecteur in r["rois"]]
  dicoimage={}
  compteur += 1

  if visualisation =="oui" :
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'])

  for i in range(len(r["rois"])) :
    vecteur_region = [str(coord) for coord in listeobjets[i]]
    classe = class_names[r["class_ids"][i]]
    score = str(r["scores"][i])
    dicoobjet = {"regions":vecteur_region, "classe" : classe, "score":score}
    dicoimage["objet"+str(i)] = dicoobjet
  dicocorpus[nomfichier] = dicoimage


with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "w") as dic :
  json.dump(dicocorpus, dic)

print("Regions of detected illustrations were saved in {}".format(os.path.join(dossier_corpus, "annotations_illustration.json")))


In [ ]:
import json

#@markdown ## Illustrations extraction
#@markdown #### Saves the detected illustrations in jpeg files
#@markdown ---

#@markdown ### Enter a destination file :
destination_file = "" #@param {type:"string"}
#@markdown Path example:
#@markdown /content/drive/My Drive/magazines/extractions/


chemin_destination = destination_file
if not chemin_destination :
   chemin_destination = os.path.join(dossier_corpus,"illustrations_extraites")

if not os.path.exists(chemin_destination):
  os.mkdir(chemin_destination)

with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "r") as mondic :
  dicocorpus2 = json.load(mondic)

print("Launching extractions")
for nomfichier in dicocorpus2 :
  cheminfichier = os.path.join(dossier_corpus, nomfichier)
  compteur = 0

  image = cv2.imread(cheminfichier)
  for objet in dicocorpus2[nomfichier].values():
    y1 = int(objet["regions"][0])
    y2 = int(objet["regions"][2])
    x1 = int(objet["regions"][1])
    x2 = int(objet["regions"][3])
    imagecropped = image[y1:y2, x1:x2] # [Y1:Y2, X1:X2] Le vecteur est sous forme [Y1 X1 Y2 X2]
    #cv2.imwrite("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites/"+nomfichier+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg", imagecropped)
    cv2.imwrite(os.path.join(chemin_destination,nomfichier+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg"), imagecropped)
    chemin_sortie = os.path.join(chemin_destination,"".join(nomfichier.split(".")[:-1])+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg") 
    cv2.imwrite(chemin_sortie, imagecropped)
    compteur += 1

print("Illustrations extracted in {}".format(chemin_destination))


